# OpenCALM SageMaker Inference with CTranslate2

[Open CALM](https://huggingface.co/cyberagent/open-calm-7b) を CTranslate2 で高速化し SageMaker でデプロイするサンプルコード。

検証は SageMaker Studio Notebook で ml.m5.4xlarge 上で PyTorch 2.0.0 Python 3.10 CPU Optimized コンテナで行いました。

In [29]:
!pip install "sagemaker>=2.143.0" -U
!pip install ctranslate2 transformers torch

  Using cached sagemaker-2.167.0-py2.py3-none-any.whl
  Using cached attrs-23.1.0-py3-none-any.whl (61 kB)
  Using cached PyYAML-6.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (682 kB)
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 5.4.1
    Uninstalling PyYAML-5.4.1:
      Successfully uninstalled PyYAML-5.4.1
  Attempting uninstall: attrs
    Found existing installation: attrs 22.2.0
    Uninstalling attrs-22.2.0:
      Successfully uninstalled attrs-22.2.0
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.153.0
    Uninstalling sagemaker-2.153.0:
      Successfully uninstalled sagemaker-2.153.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.27.132 requires PyYAML<5.5,>=3.10, but you have pyyaml 6.0 which is incompatible.
  Using cached ctrans

In [9]:
import sagemaker, boto3, json
from sagemaker import get_execution_role
from sagemaker.pytorch.model import PyTorchModel
from sagemaker.huggingface import HuggingFace

role = get_execution_role()
region = boto3.Session().region_name
sess = sagemaker.Session()
bucket = sess.default_bucket()

sagemaker.__version__

'2.167.0'

## Convert Model

モデルを CTranslate2 に最適化された形式に変換します。この処理はメモリを大きく利用するため十分なインスタンスサイズを選択してください。検証は m5.4xlarge で行いました。

In [21]:
!ct2-transformers-converter -h

usage: ct2-transformers-converter [-h] --model MODEL
                                  [--activation_scales ACTIVATION_SCALES]
                                  [--copy_files COPY_FILES [COPY_FILES ...]]
                                  [--revision REVISION] [--low_cpu_mem_usage]
                                  [--trust_remote_code] --output_dir
                                  OUTPUT_DIR [--vocab_mapping VOCAB_MAPPING]
                                  [--quantization {int8,int8_float16,int16,float16}]
                                  [--force]

options:
  -h, --help            show this help message and exit
  --model MODEL         Name of the pretrained model to download, or path to a
                        directory containing the pretrained model. (default:
                        None)
  --activation_scales ACTIVATION_SCALES
                        Path to the pre-computed activation scales. Models may
                        use them to rescale some weights to smooth the
 

In [4]:
!rm -rf scripts/model
!ct2-transformers-converter --low_cpu_mem_usage --model cyberagent/open-calm-7b --quantization int8 --output_dir scripts/model

Loading checkpoint shards: 100%|██████████████████| 2/2 [00:10<00:00,  5.21s/it]


In [5]:
!ls -l scripts/model

total 6723888
-rw-r--r-- 1 root root        129 Jun 21 16:06 config.json
-rw-r--r-- 1 root root 6882287890 Jun 21 16:06 model.bin
-rw-r--r-- 1 root root    2963299 Jun 21 16:05 vocabulary.json


## Package and Upload Model

In [6]:
!apt update -y
!apt install pigz -y

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal InRelease [265 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:4 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [2400 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal/restricted amd64 Packages [33.4 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal/multiverse amd64 Packages [177 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal/universe amd64 Packages [11.3 MB] 
Get:9 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2803 kB]
Get:10 http://security.ubuntu.com/ubuntu focal-security/multiverse amd64 Packages [28.5 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1064 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal/main amd64 Packages [1275 kB]    
Get:13 http://archive.ubuntu.com/ubun

In [7]:
%cd scripts
# !tar -czvf ../package.tar.gz *
!tar cv ./ | pigz -p 8 > ../package.tar.gz # 8 並列でアーカイブ
%cd -

/root/aws-ml-jp/tasks/generative-ai/text-to-text/fine-tuning/instruction-tuning/CTranslate2/scripts
./
./package.tar.gz
./.ipynb_checkpoints/
./model/
./model/vocabulary.json
./model/model.bin
./model/config.json
./code/
./code/requirements.txt
./code/.ipynb_checkpoints/
./code/.ipynb_checkpoints/inference-checkpoint.py
./code/.ipynb_checkpoints/requirements-checkpoint.txt
./code/inference.py
/root/aws-ml-jp/tasks/generative-ai/text-to-text/fine-tuning/instruction-tuning/CTranslate2


In [10]:
model_path = sess.upload_data('package.tar.gz', bucket=bucket, key_prefix=f"OpenCALM-Inference-CTranslate2")
model_path

's3://sagemaker-us-west-2-867115166077/OpenCALM-Inference-CTranslate2/package.tar.gz'

## Deploy Model

In [25]:
from sagemaker.serializers import JSONSerializer

endpoint_name = "OpenCALM-Inference-CTranslate"

huggingface_model = PyTorchModel(
    model_data=model_path,
    framework_version="2.0",
    py_version='py310',
    role=role,
    name=endpoint_name,
    env={
        "model_params": json.dumps({
            "tokenizer": "cyberagent/open-calm-1b",
            "model": "model",
            "prompt_input": "システム: {input}ユーザー: {instruction}<NL>システム: ",
            "prompt_no_input": "ユーザー: {instruction}<NL>システム: "
        }),
        "SAGEMAKER_MODEL_SERVER_TIMEOUT": "3600"
    }
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type='ml.g5.xlarge',
    endpoint_name=endpoint_name,
    serializer=JSONSerializer(),
)

-------------!

## Inference

In [30]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

endpoint_name = "OpenCALM-Inference-CTranslate"

predictor_client=Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sess,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer()
)
data = {
    "instruction": """ヴァージン・オーストラリアはいつから運航を開始したのですか？完結に答えてください。""".replace("\n", "<NL>"),  # システム
    "input": """ヴァージン・オーストラリア航空（Virgin Australia Airlines Pty Ltd）の商号で、オーストラリアを拠点とする航空会社です。ヴァージン・ブランドを使用する航空会社の中で、保有機材数では最大の航空会社である。2000年8月31日にヴァージン・ブルーとして、2機の航空機で単一路線で運航を開始した[3]。2001年9月のアンセット・オーストラリアの破綻後、突然オーストラリア国内市場の大手航空会社としての地位を確立した。その後、ブリスベン、メルボルン、シドニーをハブとして、オーストラリア国内の32都市に直接乗り入れるまでに成長した[4]。""".replace("\n", "<NL>"),  # ユーザー
    "max_new_tokens": 64,
    "sampling_temperature": 0.3,
    "stop_ids": [0, 1],
}
response = predictor_client.predict(
    data=data
)
print(response.replace("<NL>", "\n"))

ヴァージン・オーストラリア航空は、2000年8月31日に、アンセット・オーストラリアとヴァージン・ブルーという2つの航空会社によって設立された。アンセット・オーストラリアは、2000年8月31日に、アンセット・オーストラリアとヴァージン・ブルーという2つの航空会社によって設立された。アンセット・


## Benchmark

1.36 s ± 320 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
%timeit response = predictor_client.predict(data=data)

1.36 s ± 985 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
from tqdm import tqdm
import pandas as pd
import re
df = pd.read_json('/root/LLM/AutoModel/data/aio_02_dev_v1.0.jsonl', orient='records', lines=True)

In [81]:
%%time

def inference(instruction, input):
    data = {
        "instruction": instruction,
        "input": input,
        "max_new_tokens": 8,
        "sampling_temperature": 0,
        # "do_sample": False,
        # "sampling_topk": 500,
        # "sampling_topp": 0.95,
        # "beam_size": 5,
        # "pad_token_id": 1,
        # "bos_token_id": 0,
        # "eos_token_is": 0,
        "repetition_penalty": 1.05,
        "stop_ids": [1, 0],
    }
    response = predictor_client.predict(
        data=data
    )
    return response



# Zero Shot
correct = 0
for idx, row in df.iterrows():
    prompt = "日本語のクイズに答えてください。" + row['question'] + "答えは「"
    # print(prompt)
    result = inference("", prompt)
    # print(result)
    result = prompt + result
    try:
        result = re.findall("「(.*?)」", result)[-1]
    except IndexError:
        result = result
        # print("longer output:", result)
    result = re.sub(r'[(].*[)]', "", result)
    if result in row['answers']:
        correct += 1
    else:
        print(result, row['answers'])
print(correct, "/", len(df))

2のポーズ1 ['ジェット団']
イの釣り魚 ['コマイ']
ミサイルの“ミ” ['START', '新START']
ニュートンの頭脳 ['ニュートン', 'アイザック・ニュートン']
唐の文化 ['天平文化']
野球のリーグ ['アメリカンリーグ', 'アメリカン']
茶道 ['華道']
サウンド・オブ”クリスマス ['サラウンド', 'surround']
の ['N型半導体', 'N型', 'N']
の ['しょうゆ']
の ['ラストベルト', 'ラスト']
将棋の「駒 ['天童市', '天童']
東京大学の文学部 ['医学部']
バイト ['1024', '1024バイト']
千利休 ['村田珠光']
6 ['4人', '4']
クイズの「Q ['23時', '23']
高城の“ちゃん” ['佐々木彩夏']
の ['げっ歯類']
タングツ ['早口言葉']
クイズの“正解” ['昭和基地']
話し始めるやいなや ['開口一番']
コリオレンの仮面 ['マクベス']
机の「中 ['空論', '机上の空論']
サントリーサンゴリアス ['グリーンロケッツ']
ト長調の『ラ』の音 ['ニ長調']
御免 ['版籍奉還']
司馬懿 ['司馬遷']
大腸のABC ['IBS']
グランドセントラル ['エンパイア・ステート・ビルディング', 'エンパイア・ステート・ビル', 'エンパイアステートビル']
小説新潮 ['君たちはどう生きるか']
初月 ['明暦の大火']
日本語のクイズに答えてください。椋鳩十の童話『大造じいさんとガン』に登場する、ガンの群れの頭領の名前は何?答えは「大造じいさんの“お月さま ['残雪', 'ざんせつ']
南極 ['静かの海', '静か']
桶合戦 ['今川仮名目録']
、 ['虎の穴']
雨の七夕 ['蝉時雨']
トンボの“ミ” ['アメンボ']
。 ['七歩の才']
の ['シンコペーション']
大政 ['慶長']
子 ['獅子座']
Cのアルファベット ['G']
将棋の朝日 ['産経新聞社', '産業経済新聞社', '産経新聞']
アスタキサンチン ['ホタルイカ']
  ['鳥海', '鳥海山']
の、その ['獅子身中の虫', '獅子身中']
それは、これは ['来た']
のれん ['アロハシャツ']
花占い ['スイセン'

In [82]:
%%time
# from multiprocessing import Pool
import concurrent
from concurrent.futures import ThreadPoolExecutor

def process(question):
    prompt = "日本語のクイズに答えてください。" + question + "答えは「"
    result = inference("", prompt)
    # print(result)
    return result

# pool = Pool()                         # Create a multiprocessing Pool
# pool.map(process, df['question'])
threads = 4
with ThreadPoolExecutor(max_workers=threads) as executor:
    result = {executor.submit(process, question) for question in df['question']}
    for future in concurrent.futures.as_completed(result):
        try:
            data = future.result()
            print(data)
        except Exception as e:
            print('Looks like something went wrong:', e)

2のポーズ1」3.

唐の文化」
「日本史のクイズ
ニュートンの頭脳」
「クイズの
イの釣り魚」
「クイズ
ミサイルの“ミ”」
「
野球のリーグ」
「このボール
茶道」
「この俳句の“
サウンド・オブ”クリスマス」
の
の」
「数学クイズの解答
の」の漢字を使った料理レシピや、
の」
クイズの正解と、
将棋の「駒」と、その
東京大学の文学部」
クイズの答えの質問
キロ」の「バイト(B)」
千利休」
クイズの「
6」です。テニスのダブルスペ
クイズの「Q」と、その
高城の“ちゃん”」

の」
「このクイズの答え、
タングツ」
「アタマ
クイズの“正解”」
「
このクイズの、最初の“や”
コリオレンの仮面」
「
机の「中」の意味と、
サントリーサンゴリアス」
の
ト長調の『ラ』の音」
御免」
「三の亥
司馬懿」
「クイズの
大腸のABC」
「腸年齢
グランドセントラル」の「ポイント・オ
小説新潮」で原作が連載中の
初月」です。
次の問題
大造じいさんの“お月さま
南極」
クイズの「解答と
桶合戦」
「の、の漢字
、」
『スター・ウォーズ/
雨の七夕」
「お月
トンボの“ミ”」
クイズ
、」を、「。」。
「
の」
このクイズの答えは、
大政」
「の・曜日
子」の「星占い・星座
Cのアルファベット」
クイズの答え、
将棋の朝日」と「囲碁の日
アスタキサンチン」
「の
 」。
。ちなみに、出羽...
の、その」
「このクイズ
それは、これは」答えは次のはは
のれん」
「お寿司の日
花占い」
“の”の漢字
ディフェンダーの“の”」
10のマイナスの法則」
「数学
トリプトファン」
このクイズの
また君に恋をして」に他なりません
の」
「このクイズ、正解
ネギ」
「このクイズの“
勝つ」になる。 使う、という
ノーベル文学賞」
「このクイズの
のりの佃煮」
今週の
水素」
クイズの順番、ルール
、、「「」は(の)
聖書の「モーセ」と、
赤の「お」と、白
コーラのジュース」
「このドリンク
、」
「。」、、「」。
ジュリエットの衛星」
「クイズ
“の”と答え、その「
 最初の、の」である。

ヒマワリ」
「のぎ
ドイツ語」
クイズの順番や、
ゴールの“枠”」
クイズ
仏教」
「アフラ・マズ
“の”」
「クイズマジック
1チーム、2人」です。
の」の漢字を使った漢字クイズの問題の解


In [32]:
N = 3.6
D = 300
L = 406.4 / (N ** 0.34) + 410.7 / (D ** 0.28) + 1.69
print(L)
N = 7
D = 300
L = 406.4 / (N ** 0.34) + 410.7 / (D ** 0.28) + 1.69
print(L)

347.7653820837083
294.5637073809692


In [39]:
# 7B
print(1.212 * 3 / 60)
print(1.006 * 3 / 60)
print(0.526 * 6 / 60)
# 3B
print(1.006 * 1.5 / 60)

0.0606
0.0503
0.0526
0.02515


In [42]:
print(3/60)
print(1.5 / 60)

0.05
0.025


In [31]:
prompts = ["日本語のクイズに答えてください。" + question + "答えは「" for question in df['question']]

def inference(prompts):
    data = {
        "instruction": "",
        "input": prompts,
        "max_new_tokens": 8,
        "sampling_temperature": 0,
        "repetition_penalty": 1.05,
        "stop_ids": [1, 0],
    }
    response = predictor_client.predict(
        data=data
    )
    return response

In [ ]:
%%time

# Zero Shot
correct = 0
batch_size = 50
for idx in range(0, len(prompts), batch_size):
    results = inference(prompts[idx:idx+batch_size])
    # print(result)
    for j in range(len(results)):
        result = prompts[idx + j] + results[j]
        try:
            result = re.findall("「(.*?)」", result)[-1]
        except IndexError:
            result = result
            # print("longer output:", result)
        result = re.sub(r'[(].*[)]', "", result)
        if result in df['answers'][idx + j]:
            correct += 1
        else:
            print(result, df['answers'][idx + j])
print(correct, "/", len(df))

ジェッツ ['ジェット団']
カンカイ ['コマイ']
SALT ['START', '新START']
ア・リーグ優勝決定戦 ['アメリカンリーグ', 'アメリカン']
生け花 ['華道']
3Dサウンド ['サラウンド', 'surround']
おでん ['しょうゆ']
バイソン・バレー ['ラストベルト', 'ラスト']
文学部 ['医学部']
1キロバイト=1000バイト ['1024', '1024バイト']
千利休 ['村田珠光']
ゴールデンアワー ['23時', '23']
百田夏菜子 ['佐々木彩夏']
齧歯目 ['げっ歯類']
Tang Tiger ['早口言葉']
一文笛 ['開口一番']
シミュレーション ['空論', '机上の空論']
日本語のクイズに答えてください。NECが運営するスポーツチームで、女子バレーボールV・プレミアリーグのチーム名はレッドロケッツですが、ラグビートップリーグのチーム名は何?答えは「神戸製鋼コベルコスティーラーズ ['グリーンロケッツ']
ハ長調 ['ニ長調']
廃藩置県 ['版籍奉還']
The MIT Tower ['エンパイア・ステート・ビルディング', 'エンパイア・ステート・ビル', 'エンパイアステートビル']
十五少年漂流記 ['君たちはどう生きるか']
カッパ ['残雪', 'ざんせつ']
海 ['静かの海', '静か']
ミスターXアカデミー ['虎の穴']
セミ鳴く雨 ['蝉時雨']
オニヤンマ ['アメンボ']
七 ['七歩の才']
カノン ['シンコペーション']
水瓶座 ['獅子座']
ト音記号 ['G']
朝日新聞社 ['産経新聞社', '産業経済新聞社', '産経新聞']
アオリイカ ['ホタルイカ']
蚊 ['獅子身中の虫', '獅子身中']
ご苦労様です ['来た']
アネモネ ['スイセン']
フアンフラン ['アラ']
素数 ['完全数']
津軽海峡冬景色 ['津軽海峡・冬景色']
パクチー ['セロリ']
相かる ['運命戦']
ノーベル平和賞 ['テンプルトン賞']
近衛 ['鷹司', '鷹司家']
王様の家来になる ['白羽の矢を立てる', '白羽の矢']
カミソリ ['サインポール']
ウォッカ ['トマトジュース', 'トマト']
お小遣い ['あがり']
日本語

## Delete Endpoint

In [24]:
predictor_client.delete_model()
predictor_client.delete_endpoint()